<a href="https://colab.research.google.com/github/hawc2/Text-Analysis-with-Python/blob/master/FinBERT_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FinBERT Notebook for Todd Schifeling's research

This notebooks shows how to train and use the FinBERT pre-trained language model for financial sentiment analysis.

It was not designed to be used with Google Colab, so alterations listed below need to be made. I've finished some preliminary setup for installing packages and downloading data/models, but it isn't finished and there may be a better approach.

# Relevant Resources

Github Repo: https://github.com/ProsusAI/finBERT

# Steps to Set-Up

I've gotten pretty far, but I've encountered a couple confusions.

The Models section in Github is not clear about where to put which models, and which config.json file to use. The main error I keep getting, which worries me may be a development issue, is the following, from the Configuring Training Parameters section: 

Can't load config for '/content/models/language_model/finbertTRC2'. Make sure that:

- '/content/models/language_model/finbertTRC2' is a correct model identifier listed on 'https://huggingface.co/models'

- or '/content/models/language_model/finbertTRC2' is the correct path to a directory containing a config.json file


# Contacting Developer

It may be worth contacting the developer and ask them for help setting up a Colab environment for this script


# Set-Up Colab Notebook

In [ ]:
!pip install oblib
!pip install scikit-learn
!pip install spacy
!pip install torch
!pip install textblob
!pip install transformers

     |████████████████████████████████| 30.7MB 107kB/s 
     |████████████████████████████████| 5.8MB 42.2MB/s 
  Created wheel for enum-compat: filename=enum_compat-0.0.2-cp36-none-any.whl size=1290 sha256=c9cbee887cc5e5192d17b6f55ea2b509f68b9f2d4f4072e1bde1d07ff5c0daa9
  Stored in directory: /root/.cache/pip/wheels/b1/69/f4/229af6a49beece0f688c9c73d9188769b89e698361d21ce96a
  Created wheel for validators: filename=validators-0.12.4-cp36-none-any.whl size=15010 sha256=f6bf63a53895821128613b2804ba2e3ae01541f782bbd2538af51c5588856bac
  Stored in directory: /root/.cache/pip/wheels/6c/aa/81/0ee6b1a0d69ab3db36228d3e621691710a13ed9bb072b6e565
  Created wheel for jsondiff: filename=jsondiff-1.1.2-cp36-none-any.whl size=6466 sha256=33271510f98602d3e61d988685401f93d8706064a9a0ec976349aee9942dbebf
  Stored in directory: /root/.cache/pip/wheels/46/5f/86/11c6b72b064888e80b98bfcbcdaf2a83517a8cf8f2bb2a3227
Successfully built enum-compat validators jsondiff
  Found existing installation: lxml 4.2.6


In [ ]:
!git clone https://github.com/ProsusAI/finBERT
%cd finBERT

Cloning into 'finBERT'...
remote: Enumerating objects: 146, done.
remote: Counting objects: 100% (146/146), done.
remote: Compressing objects: 100% (93/93), done.
remote: Total 146 (delta 67), reused 121 (delta 50), pack-reused 0
Receiving objects: 100% (146/146), 54.65 KiB | 10.93 MiB/s, done.
Resolving deltas: 100% (67/67), done.
/content/finBERT


In [ ]:
from pathlib import Path
import shutil
import os
import logging
import sys
sys.path.append('..')

from textblob import TextBlob
from pprint import pprint
from sklearn.metrics import classification_report

from transformers import AutoModelForSequenceClassification

from finbert.finbert import *
import finbert.utils as tools

%load_ext autoreload
%autoreload 2

project_dir = Path.cwd().parent
pd.set_option('max_colwidth', -1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


# Import Models and Datasets

In [ ]:
!mkdir models
!mkdir models/sentiment/
!mkdir models/language_model/
!mkdir models/classifier_model/
!mkdir data
!mkdir data/sentiment_data


'/content/finBERT'

In [ ]:
!wget https://prosus-public.s3-eu-west-1.amazonaws.com/finbert/language-model/pytorch_model.bin -P /models/language_model/
!wget https://prosus-public.s3-eu-west-1.amazonaws.com/finbert/finbert-sentiment/pytorch_model.bin -P /models/sentiment/
!wget https://huggingface.co/ProsusAI/finbert/raw/main/config.json -P /models/language_models/
!wget https://huggingface.co/ProsusAI/finbert/raw/main/config.json -P /models/sentiment

--2021-01-21 15:16:46--  https://prosus-public.s3-eu-west-1.amazonaws.com/finbert/language-model/pytorch_model.bin
Resolving prosus-public.s3-eu-west-1.amazonaws.com (prosus-public.s3-eu-west-1.amazonaws.com)... 52.218.24.35
Connecting to prosus-public.s3-eu-west-1.amazonaws.com (prosus-public.s3-eu-west-1.amazonaws.com)|52.218.24.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 440477289 (420M) [application/octet-stream]
Saving to: ‘/models/language_model/pytorch_model.bin.1’

pytorch_model.bin.1 100%[===================>] 420.07M  45.4MB/s    in 8.6s    

2021-01-21 15:16:55 (49.1 MB/s) - ‘/models/language_model/pytorch_model.bin.1’ saved [440477289/440477289]

--2021-01-21 15:16:55--  https://prosus-public.s3-eu-west-1.amazonaws.com/finbert/finbert-sentiment/pytorch_model.bin
Resolving prosus-public.s3-eu-west-1.amazonaws.com (prosus-public.s3-eu-west-1.amazonaws.com)... 52.218.29.24
Connecting to prosus-public.s3-eu-west-1.amazonaws.com (prosus-public.s3

## Download FinancialPhraseBank Zip

Wget wasn't working, not sure it can be automated

Next couple cells:
1) Upload zip to Colab
2) Unzip with command
3) Run the datasets script on the Sentences_50Agree.txt in the unzipped folder

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving FinancialPhraseBank-v1.0.zip to FinancialPhraseBank-v1.0.zip
User uploaded file "FinancialPhraseBank-v1.0.zip" with length 681890 bytes


In [ ]:
!unzip FinancialPhraseBank-v1.0.zip

Archive:  FinancialPhraseBank-v1.0.zip
   creating: FinancialPhraseBank-v1.0/
  inflating: FinancialPhraseBank-v1.0/License.txt  
   creating: __MACOSX/
   creating: __MACOSX/FinancialPhraseBank-v1.0/
  inflating: __MACOSX/FinancialPhraseBank-v1.0/._License.txt  
  inflating: FinancialPhraseBank-v1.0/README.txt  
  inflating: __MACOSX/FinancialPhraseBank-v1.0/._README.txt  
  inflating: FinancialPhraseBank-v1.0/Sentences_50Agree.txt  
  inflating: FinancialPhraseBank-v1.0/Sentences_66Agree.txt  
  inflating: FinancialPhraseBank-v1.0/Sentences_75Agree.txt  
  inflating: FinancialPhraseBank-v1.0/Sentences_AllAgree.txt  


In [ ]:
!python scripts/datasets.py --data_path /content/finBERT/FinancialPhraseBank-v1.0/Sentences_50Agree.txt

scripts/datasets.py:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data = pd.read_csv(args.data_path, sep='.@', names=['text','label'])
Traceback (most recent call last):
  File "scripts/datasets.py", line 13, in <module>
    data = pd.read_csv(args.data_path, sep='.@', names=['text','label'])
  File "/usr/local/lib/python3.6/dist-packages/pandas/io/parsers.py", line 688, in read_csv
    return _read(filepath_or_buffer, kwds)
  File "/usr/local/lib/python3.6/dist-packages/pandas/io/parsers.py", line 454, in _read
    parser = TextFileReader(fp_or_buf, **kwds)
  File "/usr/local/lib/python3.6/dist-packages/pandas/io/parsers.py", line 948, in __init__
    self._make_engine(self.engine)
  File "/usr/local/lib/python3.6/dist-packages/pandas/io/parsers.py", line 1191, in _make_engine
    self._en

## Modules 

In [ ]:
logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.ERROR)

## Prepare the model

### Setting path variables:
1. `lm_path`: the path for the pre-trained language model (If vanilla Bert is used then no need to set this one).
2. `cl_path`: the path where the classification model is saved.
3. `cl_data_path`: the path of the directory that contains the data files of `train.csv`, `validation.csv`, `test.csv`.
---

In the initialization of `bertmodel`, we can either use the original pre-trained weights from Google by giving `bm = 'bert-base-uncased`, or our further pre-trained language model by `bm = lm_path`


---
All of the configurations with the model is controlled with the `config` variable. 

In [ ]:
lm_path = project_dir/'models'/'language_model'/'finbertTRC2'
cl_path = project_dir/'models'/'classifier_model'/'finbert-sentiment'
cl_data_path = project_dir/'data'/'sentiment_data'

###  Configuring training parameters

You can find the explanations of the training parameters in the class docsctrings. 

In [ ]:
# Clean the cl_path
try:
    shutil.rmtree(cl_path) 
except:
    pass

bertmodel = AutoModelForSequenceClassification.from_pretrained(lm_path,cache_dir=None, num_labels=3)


config = Config(   data_dir=cl_data_path,
                   bert_model=bertmodel,
                   num_train_epochs=4,
                   model_dir=cl_path,
                   max_seq_length = 48,
                   train_batch_size = 32,
                   learning_rate = 2e-5,
                   output_mode='classification',
                   warm_up_proportion=0.2,
                   local_rank=-1,
                   discriminate=True,
                   gradual_unfreeze=True)

404 Client Error: Not Found for url: https://huggingface.co//content/finBERT/finBERT/models/language_model/finbertTRC2/resolve/main/config.json


OSError: ignored

`finbert` is our main class that encapsulates all the functionality. The list of class labels should be given in the prepare_model method call with label_list parameter.

In [ ]:
finbert = FinBert(config)
finbert.base_model = 'bert-base-uncased'
finbert.config.discriminate=True
finbert.config.gradual_unfreeze=True

In [ ]:
finbert.prepare_model(label_list=['positive','negative','neutral'])

## Fine-tune the model

In [ ]:
# Get the training examples
train_data = finbert.get_data('train')

In [ ]:
model = finbert.create_the_model()

### [Optional] Fine-tune only a subset of the model
The variable `freeze` determines the last layer (out of 12) to be freezed. You can skip this part if you want to fine-tune the whole model.

<span style="color:red">Important: </span>
Execute this step if you want a shorter training time in the expense of accuracy.

In [ ]:
# This is for fine-tuning a subset of the model.

freeze = 6

for param in model.bert.embeddings.parameters():
    param.requires_grad = False
    
for i in range(freeze):
    for param in model.bert.encoder.layer[i].parameters():
        param.requires_grad = False

### Training

In [ ]:
trained_model = finbert.train(train_examples = train_data, model = model)

## Test the model

`bert.evaluate` outputs the DataFrame, where true labels and logit values for each example is given

In [ ]:
test_data = finbert.get_data('test')

In [ ]:
results = finbert.evaluate(examples=test_data, model=trained_model)

### Prepare the classification report

In [ ]:
def report(df, cols=['label','prediction','logits']):
    #print('Validation loss:{0:.2f}'.format(metrics['best_validation_loss']))
    cs = CrossEntropyLoss(weight=finbert.class_weights)
    loss = cs(torch.tensor(list(df[cols[2]])),torch.tensor(list(df[cols[0]])))
    print("Loss:{0:.2f}".format(loss))
    print("Accuracy:{0:.2f}".format((df[cols[0]] == df[cols[1]]).sum() / df.shape[0]) )
    print("\nClassification Report:")
    print(classification_report(df[cols[0]], df[cols[1]]))

In [ ]:
results['prediction'] = results.predictions.apply(lambda x: np.argmax(x,axis=0))

In [ ]:
report(results,cols=['labels','prediction','predictions'])

### Get predictions

With the `predict` function, given a piece of text, we split it into a list of sentences and then predict sentiment for each sentence. The output is written into a dataframe. Predictions are represented in three different columns: 

1) `logit`: probabilities for each class

2) `prediction`: predicted label

3) `sentiment_score`: sentiment score calculated as: probability of positive - probability of negative

Below we analyze a paragraph taken out of [this](https://www.economist.com/finance-and-economics/2019/01/03/a-profit-warning-from-apple-jolts-markets) article from The Economist. For comparison purposes, we also put the sentiments predicted with TextBlob.
> Later that day Apple said it was revising down its earnings expectations in the fourth quarter of 2018, largely because of lower sales and signs of economic weakness in China. The news rapidly infected financial markets. Apple’s share price fell by around 7% in after-hours trading and the decline was extended to more than 10% when the market opened. The dollar fell by 3.7% against the yen in a matter of minutes after the announcement, before rapidly recovering some ground. Asian stockmarkets closed down on January 3rd and European ones opened lower. Yields on government bonds fell as investors fled to the traditional haven in a market storm.

In [ ]:
text = "Later that day Apple said it was revising down its earnings expectations in \
the fourth quarter of 2018, largely because of lower sales and signs of economic weakness in China. \
The news rapidly infected financial markets. Apple’s share price fell by around 7% in after-hours \
trading and the decline was extended to more than 10% when the market opened. The dollar fell \
by 3.7% against the yen in a matter of minutes after the announcement, before rapidly recovering \
some ground. Asian stockmarkets closed down on January 3rd and European ones opened lower. \
Yields on government bonds fell as investors fled to the traditional haven in a market storm."

In [ ]:
cl_path = project_dir/'models'/'classifier_model'/'finbert-sentiment'
model = AutoModelForSequenceClassification.from_pretrained(cl_path, cache_dir=None, num_labels=3)

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
result = predict(text,model)

In [ ]:
blob = TextBlob(text)
result['textblob_prediction'] = [sentence.sentiment.polarity for sentence in blob.sentences]
result

In [ ]:
print(f'Average sentiment is %.2f.' % (result.sentiment_score.mean()))

Here is another example

In [ ]:
text2 = "Shares in the spin-off of South African e-commerce group Naspers surged more than 25% \
in the first minutes of their market debut in Amsterdam on Wednesday. Bob van Dijk, CEO of \
Naspers and Prosus Group poses at Amsterdam's stock exchange, as Prosus begins trading on the \
Euronext stock exchange in Amsterdam, Netherlands, September 11, 2019. REUTERS/Piroschka van de Wouw \
Prosus comprises Naspers’ global empire of consumer internet assets, with the jewel in the crown a \
31% stake in Chinese tech titan Tencent. There is 'way more demand than is even available, so that’s \
good,' said the CEO of Euronext Amsterdam, Maurice van Tilburg. 'It’s going to be an interesting \
hour of trade after opening this morning.' Euronext had given an indicative price of 58.70 euros \
per share for Prosus, implying a market value of 95.3 billion euros ($105 billion). The shares \
jumped to 76 euros on opening and were trading at 75 euros at 0719 GMT."

In [ ]:
result2 = predict(text2,model)
blob = TextBlob(text2)
result2['textblob_prediction'] = [sentence.sentiment.polarity for sentence in blob.sentences]

In [ ]:
result2

In [ ]:
print(f'Average sentiment is %.2f.' % (result2.sentiment_score.mean()))